In [56]:
#Import all libraries

from selenium import webdriver
import time
import urllib
import urllib.request
import ssl
import mysql.connector
ssl._create_default_https_context = ssl._create_unverified_context
import ast
import json
import uuid

In [25]:
def concatenate_list_data(list):
    result = ''
    for element in list:
        result += str(element)
    return result 

In [38]:
driver = webdriver.Firefox(executable_path="C:\Drivers\geckodriver.exe")
driver.maximize_window()

In [ ]:
# pip3 install mysql-connector-python

In [9]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="arun",
  database="flaskapp", 
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

In [39]:
driver.get("https://www.myntra.com/watches?f=Categories%3AWatches&p=159")
driver.set_page_load_timeout(200)    

In [ ]:
# CREATE TABLE myntra_scraper(SKU varchar(90), Product_Link varchar(1100), Title varchar(800), Category varchar(150), Product_Details varchar(5000), Specification varchar(6000), Image_Link1 varchar(1200), Image_Link2 varchar(1200));  

In [40]:
data = []
for initial in range(0,1):    
    links = []
    for product_base in driver.find_elements_by_class_name('product-base'):
        links.append( product_base.find_element_by_xpath('./a').get_attribute("href"))
        
    #50 links here... 
    
    for i in range(0, len(links)):
        print(i)
        driver.get(links[i])
        time.sleep(2)
        SKU = driver.find_element_by_class_name('supplier-styleId').get_attribute("innerHTML") 
        Product_Link = links[i]                       
        Title = driver.find_element_by_class_name('pdp-name').get_attribute("innerHTML")
        try:
            result = links[i].find('myntra.com')
            result = result + 11
            list1 = []
            while (links[i][result]!='/'):
                list1.append(links[i][result])
                result = result + 1
            Category = concatenate_list_data(list1)
            
        except:
            Category = 'NULL'
            
        
        Product_Details = driver.find_element_by_class_name('pdp-product-description-content').get_attribute("innerHTML")
        Product_Details = Product_Details.replace("<br>", ",")
        
        try:
            driver.find_element_by_class_name('index-showMoreText').click()
        except:
            pass
        
        Specification = {}
        for index_row in driver.find_element_by_class_name('index-tableContainer').find_elements_by_class_name('index-row'):
            Specification[index_row.find_element_by_class_name('index-rowKey').get_attribute("innerHTML")] =  index_row.find_element_by_class_name('index-rowValue').get_attribute("innerHTML") 
            Specification['productId'] = driver.find_element_by_class_name('supplier-styleId').get_attribute("innerHTML")
            
        Specification = str(Specification)
        #Specification = ast.literal_eval(Specification)
        
        link_tags = driver.find_elements_by_class_name('image-grid-image')
        src = []
        count=0
        for tag in link_tags:
            lop = tag.get_attribute('style')
            if 'https://assets.myntassets.com/' in lop:
                lop=lop[23:-3]
                lop = lop.replace('h_720', 'h_1440')
                lop = lop.replace('w_540', 'w_1080')
                src.append(lop)
                count= count+1
                if (count==2):
                    break
        
        Image_Link1 = src[0]
        Image_Link2 = src[1]
        
        sql = "INSERT INTO myntra_scraper (SKU, Product_Link, Title, Category, Product_Details, Specification, Image_Link1, Image_Link2) VALUES (%s, %s, %s,%s,%s,%s,%s,%s)"
        val = (SKU, Product_Link, Title, Category, Product_Details, Specification, Image_Link1, Image_Link2)
        mycursor.execute(sql, val)   #executemany
        mydb.commit()
        
        
        dict={}
        
        dict['SKU'] = SKU
        dict['Product_Link'] = Product_Link
        dict['Title'] = Title
        dict['Category'] = Category
        dict['Product_Details'] = Product_Details
        dict['Specification'] = Specification
        
        dict['Image1'] = {}
        dict['Image2'] = {}
        
        dict['Image1']['image_code'] = str(uuid.uuid4())
        dict['Image1']['image_url'] = Image_Link1
        
        dict['Image2']['image_code'] = str(uuid.uuid4())
        dict['Image2']['image_url'] = Image_Link2
        
        data.append(dict)
        
    
    no = str(initial+1)
    
    pagination_link = 'https://www.myntra.com/watches?f=Categories%3AWatches&p=' + no
    driver.get(pagination_link)
    time.sleep(4)
    
with open('data1.txt', 'w') as outfile:
    json.dump(data, outfile)

0
1
2
3


KeyboardInterrupt: 

In [ ]:
#data can be dict or list
#dump as json
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)

#load as json
with open('data.txt') as json_file:
    data = json.load(json_file)

In [57]:
import uuid
dict={}

dict['Image1'] = {}
dict['Image2'] = {}
        
dict['Image1']['image_code'] = str(uuid.uuid4())
dict['Image1']['image_url'] = 'hry'
        
dict['Image2']['image_code'] = str(uuid.uuid4())
dict['Image2']['image_url'] = 'hey'

In [58]:
print(dict)

{'Image1': {'image_code': '92c084b9-e76e-459a-a8ff-79b28ff30e63', 'image_url': 'hry'}, 'Image2': {'image_code': '73769776-c845-418a-babb-5d974a22172c', 'image_url': 'hey'}}
